<a href="https://www.kaggle.com/code/pyliraghuram/gradien-boost?scriptVersionId=144446527" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# aAnalyzing the given data

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
test.duplicated().value_counts()

In [ ]:
train.duplicated().value_counts()

In [ ]:
test.drop_duplicates(inplace=True) 
train.drop_duplicates(inplace=True) 


In [ ]:
train

In [ ]:
for i in train.columns:
    print(i)
    print('*--*'*10)
    print(train[i].value_counts(normalize = True,sort = True)*100)
    

# cleaning the data 

In [ ]:
train =train.drop(['Id','Street', 'Utilities','LandSlope','Condition2','RoofMatl','CentralAir','LowQualFinSF','3SsnPorch','ScreenPorch','PoolArea','MiscVal'], axis=1)
test =test.drop(['Id','Street', 'Utilities','LandSlope','Condition2','RoofMatl','CentralAir','LowQualFinSF','3SsnPorch','ScreenPorch','PoolArea','MiscVal'], axis=1)

In [ ]:
for df in (train, test):
    df["n_bathrooms"] = df["BsmtFullBath"] + (df["BsmtHalfBath"]*0.5) + df["FullBath"] + (df["HalfBath"]*0.5)
    df["area_with_basement"]  = df["GrLivArea"] + df["TotalBsmtSF"]
    

In [ ]:
for i in train.columns :
    print('-*-*'*10)
    print(i)
    print(train[i].isnull().sum())
    print('___+++__'*8)

In [ ]:
sns.heatmap(test.corr())

In [ ]:
sns.heatmap(train.corr())

# making the data

In [ ]:
X_train = train.select_dtypes(include=['number']).copy()
X_train = X_train.drop(['SalePrice'], axis=1)
y_train = train["SalePrice"]
X_test  = test.select_dtypes(include=['number']).copy()

In [ ]:
X_train

In [ ]:
for i in X_train.columns :
    print('-*-*'*10)
    print(i)
    print(X_train[i].isnull().sum())
    print('___+++__'*8)

In [ ]:
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [ ]:
for i in X_train.columns :
    print('-*-*'*10)
    print(i)
    print(X_train[i].isnull().sum())
    print('___+++__'*8)

In [ ]:
for i in X_test.columns :
    print('-*-*'*10)
    print(i)
    print(X_test[i].isnull().sum())
    print('___+++__'*8)

# feature scaling

In [ ]:
import xgboost as xg
from sklearn.preprocessing import RobustScaler


rrs =RobustScaler()
X_train= rrs.fit_transform(X_train)
X_test = rrs.transform(X_test)

# Gradient boost with grid search cv

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

gb = GradientBoostingRegressor()

In [ ]:
warnings.resetwarnings()

In [ ]:
# gs = GridSearchCV(gb,cv = 4,param_grid  = {'max_depth' : [3,4],'min_samples_split': [2,4]
# ,'min_samples_leaf': [2, 4],'learning_rate': [0.01, 0.1]})
# gs.fit(X_train, y_train)

# print("Best Parameters:", gs.best_params_)
# print("Best Score:", gs.best_score_)

In [ ]:
gs = GridSearchCV(gb,cv = 4,param_grid  = {'max_depth' : [3,4],'min_samples_split': [2,4]
,'min_samples_leaf': [2, 4],'learning_rate': [0.01, 0.1]},error_score='raise')
gs.fit(X_train, y_train)

print("Best Parameters:", gs.best_params_)
print("Best Score:", gs.best_score_)

In [ ]:
gs = GridSearchCV(gb,cv = 4,param_grid  = {'max_depth' : [3,4],'min_samples_split': [2,4]
,'min_samples_leaf': [2, 4],'learning_rate': [0.01, 0.1]})
gs.fit(X_train, y_train)

print("Best Parameters:", gs.best_params_)
print("Best Score:", gs.best_score_)

In [ ]:

gb = GradientBoostingRegressor(learning_rate= 0.1, max_depth=3,min_samples_leaf=2,min_samples_split= 2)
gb.fit(X_train,y_train)
print(gb.score(X_train,y_train)*100)

In [ ]:
prediction = gb.predict(X_test)

In [ ]:
prediction

In [ ]:
solution = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
y_test = solution['SalePrice']


In [ ]:
from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(y_test, prediction) )
print("The score is %.5f" % RMSLE )

In [ ]:
y_train.isnull().sum()

In [ ]:
y_test.isnull().sum()

In [ ]:
from sklearn.metrics import r2_score
r2_score(prediction,y_test)